In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Fraudfinder - BigQuery ML - Model training and prediction

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name=Model%20Monitoring&download_url=https%3A%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmaster%2Fnotebooks%2Fcommunity%2Fmodel_monitoring%2Fmodel_monitoring_feature_attribs.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Google Cloud Notebooks">Open in Cloud Notebook
    </a>
  </td> 
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/model_monitoring/model_monitoring_feature_attribs.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Open in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/model_monitoring/model_monitoring_feature_attribs.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

[Fraudfinder](https://github.com/googlecloudplatform/fraudfinder) is a series of labs on how to build a real-time fraud detection system on Google Cloud. Throughout the Fraudfinder labs, you will learn how to read historical bank transaction data stored in data warehouse, read from a live stream of new transactions, perform exploratory data analysis (EDA), do feature engineering, ingest features into a feature store, train a model using feature store, register your model in a model registry, evaluate your model, deploy your model to an endpoint, do real-time inference on your model with feature store, and monitor your model.

### Objective

In this notebook, Using the data in Vertex AI Feature Store, that you previously ingested data into, you will train a model using BigQuery ML, register the model to Vertex AI Model Registry, and deploy it to an endpoint for real-time prediction. 

In this tutorial, you will learn how to:

- Train a logistic regression model in BigQuery using BigQueryML
- Evaluate the model
- Test a prediction 
- Deploy to an endpoint on Vertex AI
- Make an online prediction 


This tutorial uses the following Google Cloud data analytics and ML services:

- BigQuery
- BigQuery ML
- Vertex AI Model Registry
- Vertex endpoints


The steps performed include:

- Using Python & SQL to query the public data in BigQuery
- Preparing the data for modeling
- Training a classification model using BigQuery ML and registering it to Vertex AI Model Registry
- Inspecting the model on Vertex AI Model Registry
- Deploying the model to an endpoint on Vertex AI
- Making sample online predictions to the model endpoint

### Costs 

This tutorial uses billable components of Google Cloud:

* BigQuery
* BigQuery ML
* Vertex AI


Learn about [BigQuery Pricing](https://cloud.google.com/bigquery/pricing), [BigQuery ML pricing](https://cloud.google.com/bigquery-ml/pricing), [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Load config settings

In [ ]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
config = !gsutil cat gs://{BUCKET_NAME}/config/notebook_env.py
print(config.n)
exec(config.n)

### Define constants

In [ ]:
from datetime import datetime, timedelta

In [ ]:
start_date = datetime.now() - timedelta(days=1)

In [ ]:
START_DATE_TRAIN = (
    "2022-01-31"  # consider few days for training (demo) # it should be yesterday
)
END_DATE_TRAIN = "2022-01-31"
CUSTOMERS_TABLE_NAME = f"customers_{END_DATE_TRAIN.replace('-', '')}"
TERMINALS_TABLE_NAME = f"terminals_{END_DATE_TRAIN.replace('-', '')}"

SERVING_FEATURE_IDS = {"customer": ["*"], "terminal": ["*"]}
READ_INSTANCES_TABLE = f"ground_truth_{END_DATE_TRAIN.replace('-', '')}"
READ_INSTANCES_URI = f"bq://{PROJECT_ID}.tx.{READ_INSTANCES_TABLE}"
BQ_TABLE_NAME = f"train_table_{END_DATE_TRAIN.replace('-', '')}"
TRAIN_TABLE_URI = f"bq://{PROJECT_ID}.tx.{BQ_TABLE_NAME}"
ff_public_topic_id = "projects/cymbal-fraudfinder/topics/ff-tx"

#### Payload schema

In [ ]:
PAYLOAD_SCHEMA = {
    "tx_amount": "float64",
    "customer_id_nb_tx_1day_window": "int64",
    "customer_id_nb_tx_7day_window": "int64",
    "customer_id_nb_tx_14day_window": "int64",
    "customer_id_avg_amount_1day_window": "float64",
    "customer_id_avg_amount_7day_window": "float64",
    "customer_id_avg_amount_14day_window": "float64",
    "customer_id_nb_tx_15min_window": "int64",
    "customer_id_avg_amount_15min_window": "float64",
    "customer_id_nb_tx_30min_window": "int64",
    "customer_id_avg_amount_30min_window": "float64",
    "customer_id_nb_tx_60min_window": "int64",
    "customer_id_avg_amount_60min_window": "float64",
    "terminal_id_nb_tx_1day_window": "int64",
    "terminal_id_nb_tx_7day_window": "int64",
    "terminal_id_nb_tx_14day_window": "int64",
    "terminal_id_risk_1day_window": "float64",
    "terminal_id_risk_7day_window": "float64",
    "terminal_id_risk_14day_window": "float64",
    "terminal_id_nb_tx_15min_window": "int64",
    "terminal_id_avg_amount_15min_window": "float64",
    "terminal_id_nb_tx_30min_window": "int64",
    "terminal_id_avg_amount_30min_window": "float64",
    "terminal_id_nb_tx_60min_window": "int64",
    "terminal_id_avg_amount_60min_window": "float64",
}

### Import libraries

In [ ]:
from typing import Union

import pandas as pd
from google.cloud import aiplatform as vertex_ai
from google.cloud import bigquery

pd.set_option("display.max_columns", None)

### Initialize Vertex AI and BigQuery SDKs for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
vertex_ai.init(project=PROJECT_ID, location=REGION)

### Helpers

Use a helper function for sending queries to BigQuery.

In [ ]:
# Wrapper to use BigQuery client to run query/job, return job ID or result as DF
def run_bq_query(sql: str) -> Union[str, pd.DataFrame]:
    """
    Input: SQL query, as a string, to execute in BigQuery
    Returns the query results as a pandas DataFrame, or error, if any
    """

    bq_client = bigquery.Client()

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    df = client_result.result().to_arrow().to_pandas()
    print(f"Finished job_id: {job_id}")
    return df

## Fetching feature values for model training

Let's assume now that new labels become available. The features you calculated are not align with them. In your case you will know if a transaction is a fraud or not only after some time the investigation has run and validated your predictions. Because of all that, it sounds incredibly difficult to say which set of features contains the most up to date historical information associated with the label you want to predict. And, when you are not able to guarantee that, the performance of your model would be badly affected because you serve no representative features of the data and the label from the field when the model goes live. So you need a way to get the most updated features you calculated over time before the label becomes available. In other terms, you need a time travel machine for your features with respect to your labels! 

Vertex AI Feature store allows to address this challenge by providing point-in-time lookups whose fetch the most up to date features with respect of the time label becomes available 

To fetch training data, we have to specify the following inputs to batch serving:

- a file containing a "query", with the entities and timestamps for each label
- a list of features to fetch values for
- the destination location and format


### Read-instance list

In our case, we need a csv file with content formatted like the table below:

|customer                     |terminal|timestamp                                    |
|-----------------------------|--------|---------------------------------------------|
|xxx3859                         |xxx8811    |2021-07-07 00:01:10 UTC                      |
|xxx4165                         |xxx8810    |2021-07-07 00:01:55 UTC                      |
|xxx2289                         |xxx2081    |2021-07-07 00:02:12 UTC                      |
|xxx3227                         |xxx3011    |2021-07-07 00:03:23 UTC                      |
|xxx2819                         |xxx6263    |2021-07-07 00:05:30 UTC                      |

where the column names are the name of entities in Feature Store and the timestamps represents the time an event occurred.

In [ ]:
read_instances_query = f"""
CREATE OR REPLACE TABLE {PROJECT_ID}.tx.{READ_INSTANCES_TABLE} as (
    SELECT
        raw_tx.TX_TS AS timestamp,
        raw_tx.CUSTOMER_ID AS customer,
        raw_tx.TERMINAL_ID AS terminal,
        raw_tx.TX_AMOUNT AS tx_amount,
        raw_lb.TX_FRAUD AS tx_fraud,
    FROM 
        tx.tx as raw_tx
    LEFT JOIN 
        tx.txlabels as raw_lb
    ON raw_tx.TX_ID = raw_lb.TX_ID
    WHERE
        DATE(raw_tx.TX_TS) = "2022-01-31" -- this should be the duration of the period that we backfilled for
);

"""

run_bq_query(read_instances_query)
run_bq_query(f"SELECT * FROM {PROJECT_ID}.tx.{READ_INSTANCES_TABLE} LIMIT 10")

### Get Feature Store

In [ ]:
try:
    ff_feature_store = vertex_ai.Featurestore(f"{FEATURESTORE_ID}")
    print(f"""The feature store {FEATURESTORE_ID} found!""")
except NameError:
    print(f"""The feature store {FEATURESTORE_ID} does not exist!""")

### Export a sample of data to a Bigquery using Feature store point in time capabilities. 

In this section, we will use Batch Serving of feature store to prepare a dataset for training by calling the BatchReadFeatureValues API. Batch Serving is used to fetch a large set of feature values with high throughput, typically for training a model or batch prediction. 

In [ ]:
ff_feature_store.batch_serve_to_bq(
    bq_destination_output_uri=TRAIN_TABLE_URI,
    serving_feature_ids=SERVING_FEATURE_IDS,
    read_instances_uri=READ_INSTANCES_URI,
    pass_through_fields=["tx_amount", "tx_fraud"],
)

### Inspect the feature table

As seen below, each row represents a transaction id, and the columns represent the attributes of the transaction (i.e customer_id, terminal_id, tx_amount),  aggregated behavioral features and the label (whether the transaction is fraudulent or not).

In [ ]:
sql_inspect = f"""
SELECT
    *
FROM
    `tx.{BQ_TABLE_NAME}`
LIMIT
    5
"""
run_bq_query(sql_inspect)

## BigQuery ML introduction

BigQuery ML (BQML) provides the capability to train ML tabular models, such as classification, regression, forecasting, and matrix factorization, in BigQuery using SQL syntax directly. BigQuery ML uses the scalable infrastructure of BigQuery ML so you don't need to set up additional infrastructure for training or batch serving.

Learn more about [BigQuery ML documentation](https://cloud.google.com/bigquery-ml/docs).

### Train a logistic regression model using BigQuery ML

The query below trains a logistic regression model using BigQuery ML. BigQuery resources are used to train the model.

In the `OPTIONS` parameter:
* with `model_registry="vertex_ai"`, the BigQuery ML model will automatically be [registered to Vertex AI Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/model-registry-bqml), which enables you to view all of your registered models and its versions on Google Cloud in one place.

* `vertex_ai_model_version_aliases allows you to set aliases to help you keep track of your model version ([documentation](https://cloud.google.com/vertex-ai/docs/model-registry/model-alias)).

In [ ]:
# this cell may take ~4 min to run

sql_train_model_bqml = f"""
CREATE OR REPLACE MODEL `tx.{MODEL_NAME}` 
OPTIONS(
  MODEL_TYPE="LOGISTIC_REG",
  INPUT_LABEL_COLS=["tx_fraud"],
  EARLY_STOP =TRUE,
  MIN_REL_PROGRESS=0.01,
  model_registry="vertex_ai", 
  vertex_ai_model_version_aliases=['bqml-ff', 'v1']
) AS

SELECT
  * EXCEPT(timestamp, entity_type_customer, entity_type_terminal)
FROM
   `tx.{BQ_TABLE_NAME}`
"""

print(sql_train_model_bqml)

run_bq_query(sql_train_model_bqml)

#### Inspect the model on Vertex AI Model Registry
When the model was trained in BigQuery ML, the line `model_registry="vertex_ai"` registered the model to Vertex AI Model Registry automatically upon completion.

You can view the model on the <a href="https://console.cloud.google.com/vertex-ai/models" target="_blank">Vertex AI Model Registry page</a>, or use the code below to check that it was successfully registered:

In [ ]:
model = vertex_ai.Model(model_name=MODEL_NAME)

print(model.gca_resource)

### Model evaluation

With the model created, you can now evaluate the logistic regression model. Behind the scenes, BigQuery ML automatically [split the data](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create#data_split_method), which makes it easier to quickly train and evaluate models.

In [ ]:
sql_evaluate_model = f"""
SELECT
  *
FROM
  ML.EVALUATE(MODEL tx.{MODEL_NAME})
"""

print(sql_evaluate_model)

run_bq_query(sql_evaluate_model)

These metrics help you understand the performance of the model. 

There are various metrics for logistic regression and other model types (full list of metrics can be found in the [documentation](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate#mlevaluate_output)).

### Batch prediction (with Explainable AI)

Make a batch prediction in BigQuery ML on the original training data to check the probability of a transaction to be fraudulent for transaction, as seen in the `probability` column, with the predicted label under the `predicted_tx_fraud` column.

[ML.EXPLAIN_PREDICT](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-explain-predict) has built-in [Explainable AI](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-xai-overview). This allows you to see the top contributing features to each prediction and interpret how it was computed.

In [ ]:
sql_explain_predict = f"""
SELECT
  *
FROM
  ML.EXPLAIN_PREDICT(MODEL tx.{MODEL_NAME},
    (SELECT * FROM  `tx.{BQ_TABLE_NAME}` LIMIT 10)
    )
"""

print(sql_explain_predict)

run_bq_query(sql_explain_predict)

Since the `top_feature_attributions` is a nested column, you can unnest the array ([documentation](https://cloud.google.com/bigquery/docs/reference/standard-sql/arrays)) into separate rows for each of the features. In other words, since ML.EXPLAIN_PREDICT provides the top 5 most important features, using `UNNEST` results in 5 rows per prediction:

In [ ]:
sql_explain_predict = f"""
SELECT
  tfa.*,
  predicted_tx_fraud,
  probability,
  baseline_prediction_value,
  prediction_value,
  approximation_error,
FROM
  ML.EXPLAIN_PREDICT(MODEL tx.{MODEL_NAME},
    (SELECT * FROM `tx.{BQ_TABLE_NAME}` )
    ),
  UNNEST(top_feature_attributions) as tfa
"""

print(sql_explain_predict)

run_bq_query(sql_explain_predict)

### Deploy the model to an endpoint

While BigQuery ML supports batch prediction with [ML.PREDICT](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-predict) and [ML.EXPLAIN_PREDICT](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-explain-predict), BigQuery ML is not suitable for real-time predictions where you need low latency predictions with potentially high frequency of requests.

In other words, deploying the BigQuery ML model to an endpoint enables you to do online predictions.

#### Create a Vertex AI endpoint

To deploy your model to an endpoint, you will first need to create an endpoint before you deploy the model to it.

In [ ]:
endpoint = vertex_ai.Endpoint.create(
    display_name=ENDPOINT_NAME,
    project=PROJECT_ID,
    location=REGION,
)

print(endpoint.display_name)
print(endpoint.resource_name)

#### List endpoints

List the endpoints to make sure it has successfully been created. You can also view your endpoints on the [Vertex AI Endpoints page](https://console.cloud.google.com/vertex-ai/endpoints).

In [ ]:
endpoint.list(order_by="update_time")

#### Deploy model to Vertex endpoint

With the model, you can now deploy it to an endpoint. 

In [ ]:
# deploying the model to the endpoint may take 10-15 minutes
model.deploy(endpoint=endpoint)

You can also check on the status of your model by visiting the [Vertex AI Endpoints page](https://console.cloud.google.com/vertex-ai/endpoints).

### Make online predictions to the endpoint with pub/sub -> pull subscription -> notebook -> Vertex AI endpoint
Using a sample of the training data, you can test the endpoint to make online predictions.

Below are some helper functions to help make it easier to read streaming data and do online predictions:

In [ ]:
# A function to read some sample messages (transaction) from the public Pub/Sub
def read_from_sub(project_id, subscription_path, messages=10):
    import ast

    from google.api_core import retry
    from google.cloud import pubsub_v1

    subscriber = pubsub_v1.SubscriberClient()
    subscription_path = subscriber.subscription_path(project_id, subscription_path)

    # Wrap the subscriber in a 'with' block to automatically call close() to
    # close the underlying gRPC channel when done.
    with subscriber:
        # The subscriber pulls a specific number of messages. The actual
        # number of messages pulled may be smaller than max_messages.
        response = subscriber.pull(
            subscription=subscription_path,
            max_messages=messages,
            retry=retry.Retry(deadline=300),
        )

        if len(response.received_messages) == 0:
            print("no messages")
            return

        ack_ids = []
        msg_data = []
        for received_message in response.received_messages:
            msg = ast.literal_eval(received_message.message.data.decode("utf-8"))
            print(f"Received: {msg}.")
            msg_data.append(msg)
            ack_ids.append(received_message.ack_id)

        # Acknowledges the received messages so they will not be sent again.
        subscriber.acknowledge(subscription=subscription_path, ack_ids=ack_ids)

        print(
            f"Received and acknowledged {len(response.received_messages)} messages from {subscription_path}."
        )

        return msg_data


# A function for pre-processing of payload before sending it to a Vertex AI endpoint
def preprocess(payload):
    # replace NaN's
    for key, value in payload.items():
        if value is None:
            payload[key] = 0.0
    return payload


# A function to lookup features in Vertex AI Feature Store
def features_lookup(ff_feature_store, entity, entity_ids):
    entity_type = ff_feature_store.get_entity_type(entity)
    aggregated_features = entity_type.read(entity_ids=entity_ids, feature_ids="*")
    aggregated_features_preprocessed = preprocess(aggregated_features)
    features = aggregated_features_preprocessed.iloc[0].to_dict()
    return features

You can now read some messages from Pub/Sub, preprocess it, augment it with some features from Vertex AI Feature Store, and submit to Vertex AI Endpoint for online predictions.

In [ ]:
messages = read_from_sub(
    project_id=PROJECT_ID, subscription_path="ff-tx-sub", messages=10
)

for payload_input in messages:
    print(f"The recieved payload from Pub/Sub is: {payload_input}")
    print(f"-----------------------------------------")
    payload = {}
    payload["tx_amount"] = payload_input["TX_AMOUNT"]
    # look up the customer featues from FS (wrttie by batch ingestion daily and by Dataflow in real-time)
    customer_features = features_lookup(
        ff_feature_store, "customer", [payload_input["CUSTOMER_ID"]]
    )
    payload["customer_id_nb_tx_1day_window"] = customer_features[
        "customer_id_nb_tx_1day_window"
    ]
    payload["customer_id_nb_tx_7day_window"] = customer_features[
        "customer_id_nb_tx_7day_window"
    ]
    payload["customer_id_nb_tx_14day_window"] = customer_features[
        "customer_id_nb_tx_14day_window"
    ]
    payload["customer_id_avg_amount_1day_window"] = customer_features[
        "customer_id_avg_amount_1day_window"
    ]
    payload["customer_id_avg_amount_7day_window"] = customer_features[
        "customer_id_avg_amount_7day_window"
    ]
    payload["customer_id_avg_amount_14day_window"] = customer_features[
        "customer_id_avg_amount_14day_window"
    ]
    payload["customer_id_nb_tx_15min_window"] = customer_features[
        "customer_id_nb_tx_15min_window"
    ]
    payload["customer_id_avg_amount_15min_window"] = customer_features[
        "customer_id_avg_amount_15min_window"
    ]
    payload["customer_id_nb_tx_30min_window"] = customer_features[
        "customer_id_nb_tx_30min_window"
    ]
    payload["customer_id_avg_amount_30min_window"] = customer_features[
        "customer_id_avg_amount_30min_window"
    ]
    payload["customer_id_nb_tx_60min_window"] = customer_features[
        "customer_id_nb_tx_60min_window"
    ]
    payload["customer_id_avg_amount_60min_window"] = customer_features[
        "customer_id_avg_amount_60min_window"
    ]
    # look up the terminal featues from FS (wrttie by batch ingestion daily and by Dataflow in real-time)
    terminal_features = features_lookup(
        ff_feature_store, "terminal", [payload_input["TERMINAL_ID"]]
    )
    payload["terminal_id_nb_tx_1day_window"] = terminal_features[
        "terminal_id_nb_tx_1day_window"
    ]
    payload["terminal_id_nb_tx_7day_window"] = terminal_features[
        "terminal_id_nb_tx_7day_window"
    ]
    payload["terminal_id_nb_tx_14day_window"] = terminal_features[
        "terminal_id_nb_tx_14day_window"
    ]
    payload["terminal_id_risk_1day_window"] = terminal_features[
        "terminal_id_risk_1day_window"
    ]
    payload["terminal_id_risk_7day_window"] = terminal_features[
        "terminal_id_risk_7day_window"
    ]
    payload["terminal_id_risk_14day_window"] = terminal_features[
        "terminal_id_risk_14day_window"
    ]
    payload["terminal_id_nb_tx_15min_window"] = terminal_features[
        "terminal_id_nb_tx_15min_window"
    ]
    payload["terminal_id_avg_amount_15min_window"] = terminal_features[
        "terminal_id_avg_amount_15min_window"
    ]
    payload["terminal_id_nb_tx_30min_window"] = terminal_features[
        "terminal_id_nb_tx_30min_window"
    ]
    payload["terminal_id_avg_amount_30min_window"] = terminal_features[
        "terminal_id_avg_amount_30min_window"
    ]
    payload["terminal_id_nb_tx_60min_window"] = terminal_features[
        "terminal_id_nb_tx_60min_window"
    ]
    payload["terminal_id_avg_amount_60min_window"] = terminal_features[
        "terminal_id_avg_amount_60min_window"
    ]
    payload = preprocess(payload)
    print(f"The input payload to the Vertex AI endpoint: {payload}")
    print(f"-----------------------------------------")

    result = endpoint.predict(instances=[payload])
    print(f"The prediction result: {result}")
    print(f"===============================================================")

## (DO NOT RUN) Cleaning up

In [ ]:
# endpoint[-1].undeploy_all()

In [ ]:
# delete_model_sql = f"""
# DROP MODEL `{PROJECT_ID}.{BQ_DATASET}.{BQML_MODEL_NAME}`
# """

# bq_query(delete_model_sql)